In [ ]:
# Two aims in this notebook
# 1) add N stories attribute to EUBUCCO
# 2) see if we can identify mix-use res

In [1]:
# load required libraries

import pandas as pd
import geopandas as gpd
import numpy as np
from pyproj import CRS
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import pickle
from shapely.validation import make_valid
from citymob import remove_holes, remove_invalid_geoms, remove_slivers, import_csv_w_wkt_to_gdf
import sys
from shapely.geometry import Point, Polygon

In [2]:
# the building construction file is the one that contains building group polygons
gdf_BC = import_csv_w_wkt_to_gdf('../../MSCA_data/DPE/open_data_millesime_2022-10-c_dep59_csv/csv/batiment_construction.csv',crs=2154,geometry_col='WKT')
gdf_BC

,WKT,batiment_construction_id,batiment_groupe_id,code_departement_insee,code_iris,code_commune_insee,s_geom_cstr,hauteur,altitude_sol,fictive_geom_cstr,geometry
0,"MULTIPOLYGON (((648120.2 7104434.6,648120.8 71...",BATIMENT0000000056598726-1,59183380AB0033_00cc16d59008e56,59,591831401.0,59183.0,32.0,4.0,7.0,0,"MULTIPOLYGON (((648120.200 7104434.600, 648120..."
1,"MULTIPOLYGON (((669133.0 7105045.6,669127.6 71...",BATIMENT0000000056596362-2,592604040A0746_e08c048eccb8b8b,59,592600000.0,59260.0,28.0,1.6,0.0,0,"MULTIPOLYGON (((669133.000 7105045.600, 669127..."
2,MULTIPOLYGON (((655725.5407313 7103268.5745565...,BATIMENT0000000056603258-4,59155000AB0454_c7a93be4dd46566,59,591550101.0,59155.0,14.0,8.0,3.0,0,"MULTIPOLYGON (((655725.541 7103268.575, 655728..."
3,"MULTIPOLYGON (((704215.0 7036515.8,704215.9388...",BATIMENT0000000046102397-2,uf590280373968_89dcf46922915ba,59,NaN,59028.0,60.0,5.0,NaN,0,"MULTIPOLYGON (((704215.000 7036515.800, 704215..."
4,MULTIPOLYGON (((703688.537616351 7036434.21480...,BATIMENT0000000046102442-1,uf590280012251_4fca4c9f5acb718,59,590280103.0,NaN,24.0,4.7,22.0,0,"MULTIPOLYGON (((703688.538 7036434.215, 703698..."
...,...,...,...,...,...,...,...,...,...,...,...
1012296,MULTIPOLYGON (((636715.66902653 7100960.020757...,u116pu3x94c1s84x0t61uf592720337864,uf592720337864_8b769806f01413f,59,592720101.0,59272.0,NaN,6.0,NaN,1,"MULTIPOLYGON (((636715.669 7100960.021, 636718..."
1012297,MULTIPOLYGON (((636727.716498794 7101020.10733...,u116pu9dszmpevxmhw29uf592720381593,uf592720381593_c7c05fa4d101938,59,592720101.0,59272.0,NaN,6.0,NaN,1,"MULTIPOLYGON (((636727.716 7101020.107, 636730..."
1012298,MULTIPOLYGON (((636723.992324715 7101070.20948...,u116pu9t7g4wkzfp7y8v59272000AL0617,59272000AL0617_58c51d221f4f5b6,59,592720101.0,59272.0,NaN,5.0,NaN,1,"MULTIPOLYGON (((636723.992 7101070.209, 636727..."
1012299,MULTIPOLYGON (((636918.128566155 7101093.77527...,u116puewdwjyzsb0srxe59272000AH0489,59272000AH0489_73baa030a3555ff,59,592720101.0,59272.0,NaN,5.0,NaN,1,"MULTIPOLYGON (((636918.129 7101093.775, 636919..."


In [3]:
bdt=pd.read_csv('../../MSCA_data/DPE/open_data_millesime_2022-10-c_dep59_csv/csv/batiment_groupe_bdtopo_bat.csv')
bdt

,batiment_groupe_id,code_departement_insee,l_nature,l_usage_1,l_usage_2,l_etat,hauteur_mean,altitude_sol_mean
0,59626000ZD0062_81482eddb4148c0,59,"[ ""Fort, blockhaus, casemate"" ]","[ ""Indifférencié"" ]",NaN,"[ ""En service"" ]",5,107.0
1,590430000A0992_c9e95b8a3e782f2,59,"[ ""Chapelle"" ]","[ ""Religieux"" ]",NaN,"[ ""En service"" ]",1,29.0
2,uf592520097169_45a6275f4246c8b,59,"[ ""Indifférenciée"" ]","[ ""Indifférencié"" ]",NaN,"[ ""En service"" ]",6,18.0
3,uf593080411204_978efb49cb521b7,59,"[ ""Indifférenciée"" ]","[ ""Indifférencié"" ]",NaN,"[ ""En service"" ]",4,43.0
4,uf593050411146_854550cccdbe726,59,"[ ""Indifférenciée"" ]","[ ""Résidentiel"" ]",NaN,"[ ""En service"" ]",3,15.0
...,...,...,...,...,...,...,...,...
890598,596060000E0024_d58d2e711ef3341,59,"[ ""Indifférenciée"" ]","[ ""Résidentiel"" ]","[ ""Annexe"" ]","[ ""En service"" ]",4,30.0
890599,uf596020244723_b50d302d040e6aa,59,"[ ""Indifférenciée"" ]","[ ""Résidentiel"" ]",NaN,"[ ""En service"" ]",3,29.0
890600,594000000A0248_855c239f58a786f,59,"[ ""Indifférenciée"" ]","[ ""Résidentiel"" ]",NaN,"[ ""En service"" ]",7,18.0
890601,59112000AD0393_9d1c371e536d1ca,59,"[ ""Indifférenciée"" ]","[ ""Résidentiel"" ]",NaN,"[ ""En service"" ]",6,21.0


In [4]:
dpe_log=pd.read_csv('../../MSCA_data/DPE/open_data_millesime_2022-10-c_dep59_csv/csv/dpe_logement.csv')
dpe_log_col=dpe_log.columns.to_list() # there are a lot of columns
dpe_log.head()

C:\Users\peter\AppData\Local\Temp\ipykernel_6976\2919012715.py:1: DtypeWarning: Columns (8,17,18,19,20,26,29,35,39,42,48,70,75,76,77,84,85,86,92,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  dpe_log=pd.read_csv('../../MSCA_data/DPE/open_data_millesime_2022-10-c_dep59_csv/csv/dpe_logement.csv')


,identifiant_dpe,code_departement_insee,arrete_2021,type_dpe,type_batiment_dpe,periode_construction_dpe,annee_construction_dpe,version,date_etablissement_dpe,date_reception_dpe,...,type_porte,surface_porte,u_porte,classe_inertie,deperdition_mur,deperdition_baie_vitree,deperdition_plancher_bas,deperdition_plancher_haut,deperdition_pont_thermique,deperdition_porte
0,0059L1000001@,59,0,dpe arrêté 2012 3cl immeuble,immeuble,1989-2000,1996.0,NaN,NaN,2017/05/30 00:00:00,...,NaN,NaN,NaN,Moyenne,NaN,NaN,NaN,NaN,NaN,NaN
1,0059L1000002A,59,0,dpe arrêté 2012 3cl immeuble,immeuble,1948-1974,1971.0,NaN,NaN,2017/05/31 00:00:00,...,NaN,NaN,NaN,Moyenne,NaN,NaN,NaN,NaN,NaN,NaN
2,0059L1000003,59,0,dpe arrêté 2012 3cl logement,appartement,1989-2000,1995.0,NaN,NaN,2019/07/25 00:00:00,...,NaN,NaN,NaN,Moyenne,NaN,NaN,NaN,NaN,NaN,NaN
3,0059L1000004,59,0,dpe arrêté 2012 3cl logement,appartement,1989-2000,1995.0,NaN,NaN,2019/07/25 00:00:00,...,NaN,NaN,NaN,Moyenne,NaN,NaN,NaN,NaN,NaN,NaN
4,0059L1000005D,59,0,dpe arrêté 2012 3cl logement,appartement,1989-2000,1995.0,NaN,NaN,2019/08/04 00:00:00,...,NaN,NaN,NaN,Lourde,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dpe=dpe_log.loc[:,['identifiant_dpe','arrete_2021','type_batiment_dpe','periode_construction_dpe','surface_habitable_immeuble','surface_habitable_logement','nombre_niveau_logement','nombre_niveau_immeuble']]

In [6]:
rel_bldgroup_dpe_log=pd.read_csv('../../MSCA_data/DPE/open_data_millesime_2022-10-c_dep59_csv/csv/rel_batiment_groupe_dpe_logement.csv')
rel_bldgroup_dpe_log.head()

,batiment_groupe_id,identifiant_dpe,code_departement_insee,cle_interop_adr,adresse_brut,adresse_geocodee,fiabilite_geocodage
0,59512000IN0090_b45ee30e81da3f7,1959V2014824D,59,59512_2540_00095,95 Rue D Hem 59100 Roubaix,95 Rue d’Hem 59100 Roubaix,très fiable
1,59512000IO0196_67d7fd8ce86b410,1459V2008109X,59,59512_2720_00009,9 Rue Ingres 59100 Roubaix,9 Rue Ingres 59100 Roubaix,très fiable
2,59512000IK0019_5206a20845e2bc4,1659L1003092T,59,59512_1300_00036,36 Rue Coligny 59100 Roubaix,36 Rue Coligny 59100 Roubaix,très fiable
3,59512000IK0172_1be81f80ce0da53,1559V2010882@,59,59512_1300_00009,9 Rue Coligny 59100 Roubaix,9 Rue Coligny 59100 Roubaix,très fiable
4,59512000IK0153_cfaf8654038c76e,1659V1003062R,59,59512_4250_00022,22 Rue Pascal 59100 Roubaix,22 Rue Pascal 59100 Roubaix,très fiable


In [7]:
# dpe2 is bigger than dpe, as some dpes apply to multiple building groups
dpe2=dpe.merge(rel_bldgroup_dpe_log)
dpe2

,identifiant_dpe,arrete_2021,type_batiment_dpe,periode_construction_dpe,surface_habitable_immeuble,surface_habitable_logement,nombre_niveau_logement,nombre_niveau_immeuble,batiment_groupe_id,code_departement_insee,cle_interop_adr,adresse_brut,adresse_geocodee,fiabilite_geocodage
0,0059L1000001@,0,immeuble,1989-2000,NaN,NaN,NaN,NaN,uf590090396546_7f36654d031baf2,59,59009_1347_00032,19 32 Rue Martin Luther King 59650 Villeneuve ...,32 Rue Martin Luther King 59650 Villeneuve-d'Ascq,très fiable
1,0059L1000002A,0,immeuble,1948-1974,NaN,NaN,NaN,NaN,59159000ZB0201_a00d16d14b1fca9,59,59159_0052_00009,1 9 Rue Du Lansweg 690 712 Rue De L Avenir 592...,9 Rue du Lansweg 59279 Craywick,très fiable
2,0059L1000003,0,appartement,1989-2000,NaN,NaN,NaN,NaN,59183000AM0021_727a2e9809e2df9,59,59183_1760_00025,3 25 Rue De La Verrerie 59140 Dunkerque,25 Rue de la Verrerie 59140 Dunkerque,très fiable
3,0059L1000004,0,appartement,1989-2000,NaN,NaN,NaN,NaN,59183000AM0021_727a2e9809e2df9,59,59183_1760_00025,3 25 Rue De La Verrerie 59140 Dunkerque,25 Rue de la Verrerie 59140 Dunkerque,très fiable
4,0059L1000005D,0,appartement,1989-2000,NaN,NaN,NaN,NaN,59183000AM0021_727a2e9809e2df9,59,59183_1760_00025,3 25 Rue De La Verrerie 59140 Dunkerque,25 Rue de la Verrerie 59140 Dunkerque,très fiable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310965,2292E1398791G,1,maison,avant 1948,NaN,179.17,1.0,NaN,59599000BD0506_3b8b8f80519a84e,59,59599_5400_00092,92 Rue Du Touquet 59200 Tourcoing,92 Rue Du Touquet 59200 Tourcoing,très fiable
310966,2295E0066387B,1,maison,1989-2000,NaN,149.70,1.0,NaN,59152000AK0252_30ef3077e141d33,59,59152_0745_00008,8 Rue Saint Exupery 59560 Comines,8 Rue Saint Exupery 59560 Comines,très fiable
310967,2295E0581827R,1,maison,2006-2012,NaN,85.00,2.0,NaN,59340000AI0214_1a9e9e4e9e6ba7b,59,59340_0028_00066,66 Rue Alexander Selkirk 59495 063 Leffrinckoucke,66 Rue Alexander Selkirk 59495 Leffrinckoucke,très fiable
310968,2298E0265285C,1,maison,avant 1948,NaN,124.00,2.0,NaN,uf591050400868_6dfa587115fb757,59,59105_0080_00141,141 Rue De La Chapelle 59870 9 Bouvignies,141 Rue De La Chapelle 59870 Bouvignies,très fiable


In [8]:
# dpe3 gets larger than either of it's two constituents because there are so many overlapping ids. 
# in short, each dpe actually refers to an individual building_construction, but only gets an id matching to the building_group, via the 'rel_batiment_groupe_dpe_logement' file.
# so this means that we cannot make a 1:1 mapping of dpe files to building_constructions
dpe3=dpe2.merge(gdf_BC)
dpe3.head()

,identifiant_dpe,arrete_2021,type_batiment_dpe,periode_construction_dpe,surface_habitable_immeuble,surface_habitable_logement,nombre_niveau_logement,nombre_niveau_immeuble,batiment_groupe_id,code_departement_insee,...,fiabilite_geocodage,WKT,batiment_construction_id,code_iris,code_commune_insee,s_geom_cstr,hauteur,altitude_sol,fictive_geom_cstr,geometry
0,0059L1000001@,0,immeuble,1989-2000,NaN,NaN,NaN,NaN,uf590090396546_7f36654d031baf2,59,...,très fiable,"MULTIPOLYGON (((709848.8 7063495.0,709848.6773...",BATIMENT0000000291010590-2,590090101.0,59009.0,473.0,3.3,22.0,0,"MULTIPOLYGON (((709848.800 7063495.000, 709848..."
1,0059L1000001@,0,immeuble,1989-2000,NaN,NaN,NaN,NaN,uf590090396546_7f36654d031baf2,59,...,très fiable,"MULTIPOLYGON (((709859.6 7063550.5,709859.7 70...",BATIMENT0000000291010595-1,590090101.0,59009.0,282.0,5.4,22.0,0,"MULTIPOLYGON (((709859.600 7063550.500, 709859..."
2,0059L1000001@,0,immeuble,1989-2000,NaN,NaN,NaN,NaN,uf590090396546_7f36654d031baf2,59,...,très fiable,"MULTIPOLYGON (((709851.3 7063531.1,709852.8 70...",BATIMENT0000000291010598-1,590090101.0,59009.0,219.0,3.3,21.9,0,"MULTIPOLYGON (((709851.300 7063531.100, 709852..."
3,0059L1000001@,0,immeuble,1989-2000,NaN,NaN,NaN,NaN,uf590090396546_7f36654d031baf2,59,...,très fiable,MULTIPOLYGON (((709897.058171444 7063552.76781...,BATIMENT0000000291010600-2,590090101.0,59009.0,436.0,3.3,21.8,0,"MULTIPOLYGON (((709897.058 7063552.768, 709897..."
4,0059L1000001@,0,immeuble,1989-2000,NaN,NaN,NaN,NaN,uf590090396546_7f36654d031baf2,59,...,très fiable,"MULTIPOLYGON (((710001.3 7063508.7,709998.6 70...",BATIMENT0000000291010605-1,590090101.0,59009.0,30.0,1.5,22.5,0,"MULTIPOLYGON (((710001.300 7063508.700, 709998..."


In [13]:
# note that the same batiment_construction_id's are repeatedly assigned to different identifiant_dpe and cle_interop_adr via the same batiment_groupe_id
# so we don't know which `identifiant_dpe`` and `cle_interop_adr` relate to each individual building `batiment_construction_id`
dpe3.loc[:,['identifiant_dpe','batiment_groupe_id','cle_interop_adr','batiment_construction_id']].head(22)

,identifiant_dpe,batiment_groupe_id,cle_interop_adr,batiment_construction_id
0,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010590-2
1,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010595-1
2,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010598-1
3,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010600-2
4,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010605-1
5,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010607-1
6,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291011267-1
7,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010543-1
8,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010557-2
9,0059L1000001@,uf590090396546_7f36654d031baf2,59009_1347_00032,BATIMENT0000000291010608-1


In [9]:
# so we give up for now on estimating N stories for individual building geometries, because N stories comes from the dpe data and we can't uniquely match that to individual building ids
# then we move on to doing something with building types for mix-use buildings, although I fear the same here as the bdt file with types only has building group ids, not individual buildings


1559L1003347     733
2259E0376475D    733
2259E0330972A    733
2259E0330911R    733
2259E0330671L    733
                ... 
1659L2003066V      1
1659L2003067W      1
1659L2003068X      1
1659V2002370T      1
2298E0265285C      1
Name: identifiant_dpe, Length: 284876, dtype: int64

In [20]:
bc_type=gdf_BC.merge(bdt)
bc_type.head()

,WKT,batiment_construction_id,batiment_groupe_id,code_departement_insee,code_iris,code_commune_insee,s_geom_cstr,hauteur,altitude_sol,fictive_geom_cstr,geometry,l_nature,l_usage_1,l_usage_2,l_etat,hauteur_mean,altitude_sol_mean
0,"MULTIPOLYGON (((648120.2 7104434.6,648120.8 71...",BATIMENT0000000056598726-1,59183380AB0033_00cc16d59008e56,59,591831401.0,59183.0,32.0,4.0,7.0,0,"MULTIPOLYGON (((648120.200 7104434.600, 648120...","[ ""Indifférenciée"" ]","[ ""Indifférencié"" ]",NaN,"[ ""En service"" ]",4,7.0
1,"MULTIPOLYGON (((648104.2 7104432.5,648104.7 71...",BATIMENT0000000056598724-1,59183380AB0033_00cc16d59008e56,59,591831401.0,59183.0,32.0,3.9,7.0,0,"MULTIPOLYGON (((648104.200 7104432.500, 648104...","[ ""Indifférenciée"" ]","[ ""Indifférencié"" ]",NaN,"[ ""En service"" ]",4,7.0
2,"MULTIPOLYGON (((648078.1 7104429.7,648072.1 71...",BATIMENT0000000056598731-1,59183380AB0033_00cc16d59008e56,59,591831401.0,59183.0,54.0,4.2,7.0,0,"MULTIPOLYGON (((648078.100 7104429.700, 648072...","[ ""Indifférenciée"" ]","[ ""Indifférencié"" ]",NaN,"[ ""En service"" ]",4,7.0
3,"MULTIPOLYGON (((648102.1 7104428.9,648101.7 71...",BATIMENT0000000284074632-1,59183380AB0033_00cc16d59008e56,59,591831401.0,59183.0,17.0,4.0,7.0,0,"MULTIPOLYGON (((648102.100 7104428.900, 648101...","[ ""Indifférenciée"" ]","[ ""Indifférencié"" ]",NaN,"[ ""En service"" ]",4,7.0
4,"MULTIPOLYGON (((648093.6 7104428.4,648093.5 71...",BATIMENT0000000284074633-1,59183380AB0033_00cc16d59008e56,59,591831401.0,59183.0,15.0,4.3,7.0,0,"MULTIPOLYGON (((648093.600 7104428.400, 648093...","[ ""Indifférenciée"" ]","[ ""Indifférencié"" ]",NaN,"[ ""En service"" ]",4,7.0


In [33]:
bc_type_df=pd.DataFrame(bc_type.drop(columns=['WKT','code_departement_insee','code_iris','code_commune_insee','fictive_geom_cstr','altitude_sol_mean','geometry']))

In [27]:
# we have many building groups which contain multiple buildings, and i don't think the type attribute from the bdt file necessarily relates to every single building in a building group
bc_type['batiment_groupe_id'].value_counts()

uf594560419272_54cd85991256774    733
uf594560365876_d862003c11f123d    639
uf596160250976_a2abb4d165b1810    572
uf594910325059_84f9639479cbabe    490
uf592760381753_19d72ff79c66604    486
                                 ... 
59033000AK0479_ab30958c8f9db3f      1
59033000AK0480_05494d81c35ca5e      1
59033000AK0474_58e7d9f6a03521e      1
uf590330014556_3fe1f8fa8fcf4c1      1
59272000AL0369_6f13550342a59ff      1
Name: batiment_groupe_id, Length: 890603, dtype: int64

In [31]:
# although, to be fair 96% of the building groups only have one building
sum(bc_type['batiment_groupe_id'].value_counts()==1)/len(bc_type['batiment_groupe_id'].unique())

0.9586459960274106

In [22]:
# we have unique building constructions
bc_type['batiment_construction_id'].value_counts()

BATIMENT0000000056598726-1            1
BATIMENT0000000289963192-2            1
BATIMENT0000000289963196-2            1
BATIMENT0000000289963197-2            1
BATIMENT0000000289963198-2            1
                                     ..
BATIMENT0000000288786294-1            1
BATIMENT0000000288786308-1            1
BATIMENT0000000288786317-2            1
BATIMENT0000000288786320-1            1
u116puh2jdcjg0nj2nc759272000AL0369    1
Name: batiment_construction_id, Length: 1012301, dtype: int64

In [37]:
# residential is not included here
natures=bc_type['l_nature'].drop_duplicates().to_list()

In [39]:
use1=bc_type['l_usage_1'].drop_duplicates().to_list()
use2=bc_type['l_usage_2'].drop_duplicates().to_list()

In [48]:
sum((bc_type['l_usage_2'].isna()))

848235

In [74]:
bc_type['Res_mixed']=((bc_type['l_usage_1'].str.contains('Résidentiel',na=False)) | (bc_type['l_usage_2'].str.contains('Résidentiel',na=False))) & (~bc_type['l_usage_2'].isna())
bc_type.loc[(bc_type['l_usage_1']=='[ "Résidentiel" ]') & (bc_type['l_usage_2']=='[ "Annexe" ]'),'Res_mixed']=False
bc_type_df=pd.DataFrame(bc_type.drop(columns=['WKT','code_departement_insee','code_iris','code_commune_insee','fictive_geom_cstr','altitude_sol_mean','geometry']))

In [75]:
bc_type['Res_mixed'].value_counts()

False    947270
True      65031
Name: Res_mixed, dtype: int64

In [53]:
# now merge with eubucco stock
bldg=gpd.read_file('../../MSCA_data/BuildingsDatabase/clips/eubucco_lille.shp')
bldg.head()

,fid,id,height,age,type,id_source,type_sourc,geometry
0,19439901.0,v0.1-FRA.7.2.5.19_1-4528,4.5,NaN,non-residential,BATIMENT0000000288165364,Commercial et services,"POLYGON ((3828709.383 3069743.102, 3828734.808..."
1,19442319.0,v0.1-FRA.7.2.5.19_1-6946,3.3,NaN,None,BATIMENT0000000288165365,Indifférencié,"POLYGON ((3828741.619 3069724.755, 3828755.805..."
2,19439903.0,v0.1-FRA.7.2.5.19_1-4530,0.0,2007.0,residential,BATIMENT0000000288165418,Résidentiel,"POLYGON ((3829149.498 3069720.967, 3829159.681..."
3,19323988.0,v0.1-FRA.7.2.5.13_1-1192,0.0,2006.0,residential,BATIMENT0000000288165415,Résidentiel,"POLYGON ((3829148.454 3069684.635, 3829164.402..."
4,19323943.0,v0.1-FRA.7.2.5.13_1-1147,4.4,1985.0,residential,BATIMENT0000000288165338,Résidentiel,"POLYGON ((3829195.216 3069659.877, 3829195.292..."


In [54]:
bldg.merge(bc_type_df,left_on='id_source',right_on='batiment_construction_id')

,fid,id,height,age,type,id_source,type_sourc,geometry,batiment_construction_id,batiment_groupe_id,s_geom_cstr,hauteur,altitude_sol,l_nature,l_usage_1,l_usage_2,l_etat,hauteur_mean,Res_mixed


In [56]:
eub_ids=bldg['id_source']
bc_ids=bc_type['batiment_construction_id']

In [64]:
# merge with batiment_construction_id removing the last two characteris '-N
bc_ids_short=bc_type['batiment_construction_id'].str.slice(stop=-2)

In [77]:
bc_type_df['bc_ids_short']=bc_type_df['batiment_construction_id'].str.slice(stop=-2)
# we lose >30% of buildings here from EUBUCCO because they don't have a matching id. I suspect it is buildings which are not in the BDNBD data probably because they've never had a DPE.
# to get a comprehensive mapping, we might need to go back to the original source of these type labels (BDTOPO i guess)
bldg_type=bldg.merge(bc_type_df,left_on='id_source',right_on='bc_ids_short')
bldg_type_df=pd.DataFrame(bldg_type.drop(columns='geometry'))

In [78]:
bldg_type.loc[(bldg_type['l_usage_1']=='[ "Résidentiel" ]') & (bldg_type['l_usage_2']=='[ "Annexe" ]'),'Res_mixed'].value_counts()

False    25143
Name: Res_mixed, dtype: int64

In [86]:
bldg_type['type'].value_counts()

residential        265118
non-residential     47030
Name: type, dtype: int64

In [79]:
# about 7% of buildings here are marked as residential mixed use, or over 8% of residential buildings.
# i will incorporate this approach in the methods for calculating m2 for the purpose of scaling res energy and emissions, noting however that we may still miss many buildings (~30%) which could be mixed use, but are not included in the BDNB bdtopo data
bldg_type['Res_mixed'].value_counts()

False    327373
True      22086
Name: Res_mixed, dtype: int64

In [82]:
# these are the buildings that were not matched with the bdt type dat. there are many residential buildings here
bldg.loc[~bldg['id'].isin(bldg_type['id']),'type'].value_counts()

non-residential    32016
residential        20503
Name: type, dtype: int64

In [83]:
bldg.loc[~bldg['id'].isin(bldg_type['id']),'type_sourc'].value_counts()

Indifférencié             120590
Annexe                     24750
Résidentiel                20503
Commercial et services      5237
Industriel                  1016
Agricole                     502
Sportif                      368
Religieux                    143
Name: type_sourc, dtype: int64

In [84]:
bldg.loc[~bldg['id'].isin(bldg_type['id']),]

,fid,id,height,age,type,id_source,type_sourc,geometry
0,19439901.0,v0.1-FRA.7.2.5.19_1-4528,4.5,NaN,non-residential,BATIMENT0000000288165364,Commercial et services,"POLYGON ((3828709.383 3069743.102, 3828734.808..."
1,19442319.0,v0.1-FRA.7.2.5.19_1-6946,3.3,NaN,None,BATIMENT0000000288165365,Indifférencié,"POLYGON ((3828741.619 3069724.755, 3828755.805..."
7,19442129.0,v0.1-FRA.7.2.5.19_1-6756,4.6,NaN,None,BATIMENT0000000056801702,Indifférencié,"POLYGON ((3827796.861 3069885.322, 3827800.693..."
8,19442128.0,v0.1-FRA.7.2.5.19_1-6755,3.9,NaN,None,BATIMENT0000000056801698,Indifférencié,"POLYGON ((3827821.761 3069868.882, 3827827.351..."
9,19436074.0,v0.1-FRA.7.2.5.19_1-701,4.5,NaN,None,BATIMENT0000000056801697,Indifférencié,"POLYGON ((3827821.761 3069868.882, 3827813.439..."
...,...,...,...,...,...,...,...,...
516631,19051698.0,v0.1-FRA.7.2.5.1_1-29398,1.6,NaN,None,BATIMENT0000000287750052,Indifférencié,"POLYGON ((3817486.408 3088008.629, 3817484.894..."
516632,19053089.0,v0.1-FRA.7.2.5.1_1-30789,1.9,NaN,None,BATIMENT0000000287749951,Indifférencié,"POLYGON ((3817461.767 3087979.286, 3817459.809..."
516633,19053095.0,v0.1-FRA.7.2.5.1_1-30795,2.3,NaN,None,BATIMENT0000000287749956,Indifférencié,"POLYGON ((3817453.879 3088002.724, 3817454.867..."
516634,19053097.0,v0.1-FRA.7.2.5.1_1-30797,2.2,NaN,None,BATIMENT0000000287749947,Indifférencié,"POLYGON ((3817467.100 3087978.472, 3817466.012..."
